# Plot results from GLUE evaluation

Move to root folder

In [ ]:
%cd ../..

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Load results

In [ ]:
GLUE_results = pd.read_csv("GLUE/data/results.csv")

In [ ]:
ADAPTATION_TAGS = ["avg-visual-features", 
                   "zeroed-visual-features", 
                   "imagined-visual-features",
                   "zero-image-visual-features",
                   "zeroed-visual-features",
                   "finetuned-lxmert-visual-features",
                   "finetuned-wikipedia-visual-features",
                   "trained-lxmert-scratch",
                   "trained-lxmert",
                   "trained-wikipedia",
                   "no-visual-features-finetuned-lxmert",
                   "no-visual-features-finetuned-wikipedia",
                   "no-visual-features",
                   "imagined-visual-features"
                  ]
MODEL_RENAMES = {"bert-base-uncased": "BERT-base",
                 "clipbert": "CLIP-BERT",
                 "lxmert": "LXMERT",
                 "visualbert": "VisualBERT"}

TAG_RENAMES = {"trained-lxmert": "trained-LXMERT",
             "trained-lxmert-scratch": "trained-LXMERT-scratch",
             "trained-wikipedia": "trained-Wikipedia",
             "no-visual-features": "default",
             "no-visual-features-finetuned-lxmert": "no-visual-features-finetuned-LXMERT",
             "no-visual-features-finetuned-wikipedia": "no-visual-features-finetuned-Wikipedia",
             "avg-visual-features": "avg-visual-features",
             "zero-image-visual-features": "zero-image-visual-features",
             "finetuned-lxmert-visual-features": "finetuned-LXMERT-visual-features",
             "finetuned-wikipedia-visual-features": "finetuned-Wikipedia-visual-features",
             "imagined-visual-features": "imagined-visual-features",
             "zeroed-visual-features": "zeroed-visual-features"}


def get_adaptation_tag(row):
    for tag in ADAPTATION_TAGS:
        if tag in row.model:
            return tag
    return "no-visual-features"

def get_model_name(row):
    model_name = row.model
    for tag in ADAPTATION_TAGS:
        model_name = model_name.replace("-"+tag, "")
    return model_name


HUE_ORDER = ["trained-LXMERT",
             "trained-LXMERT-scratch",
             "trained-Wikipedia",
             "default",
             "no-visual-features-finetuned-LXMERT",
             "no-visual-features-finetuned-Wikipedia",
             "avg-visual-features",
             "zero-image-visual-features",
             "zeroed-visual-features",
             "finetuned-LXMERT-visual-features",
             "finetuned-Wikipedia-visual-features",
             "imagined-visual-features"
            ]

tmp_data = GLUE_results.copy()
adaptation_tags = tmp_data.apply(get_adaptation_tag, axis=1)
model_names = tmp_data.apply(get_model_name, axis=1)

tmp_data["adaptation"] = [TAG_RENAMES[tag] for tag in adaptation_tags]
tmp_data["model"] = [MODEL_RENAMES[name] for name in model_names]

tmp_data = tmp_data.groupby(["model", "adaptation"]).mean() #take mean over concepts and relations
tmp_data = tmp_data.reset_index()

plt.figure(figsize=(3,4.7)) #(3,5)
# sns.set(rc={'text.usetex' : True})
sns.set(rc={'text.usetex' : False})
sns.set(font="serif", font_scale=0.8)
# plt.rcParams['text.usetex'] = True
plt.rcParams['text.usetex'] = False

sns.barplot(
    data=tmp_data, x='score', y='model', hue="adaptation",
    hue_order = HUE_ORDER,
    orient="h", palette="colorblind", edgecolor=".2", #width=0.9
)

plt.xlim(0.55, 0.85)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., frameon=False)

plt.xlabel("Score")
plt.ylabel("Model")

#plt.savefig("../images/GLUE_results.pgf", bbox_inches='tight')
plt.show()

In [ ]:
print(tmp_data.groupby(["model", "adaptation"]).mean().reindex(HUE_ORDER, level="adaptation").to_latex())